# 扩展思考

## 目录
- [设置](#设置)
- [基本示例](#基本示例)
- [使用扩展思考进行流式处理](#使用扩展思考进行流式处理)
- [令牌计数和上下文窗口管理](#令牌计数和上下文窗口管理)
- [理解编辑过的思考块](#理解编辑过的思考块)
- [处理错误情况](#处理错误情况)

本笔记本演示了如何通过各种示例和边缘情况使用 Claude 3.7 Sonnet 的扩展思考功能。

扩展思考为 Claude 3.7 Sonnet 提供了增强的复杂任务推理能力，同时在提供最终答案之前透明地展示其逐步的思考过程。当启用扩展思考时，Claude 会创建 `thinking` 内容块，在其中输出其内部推理。Claude 在制作最终响应之前会整合这些推理的见解。有关扩展思考的更多信息，请参阅我们的[文档](https://docs.claude.com/en/docs/build-with-claude/extended-thinking)。

## 设置

首先，让我们安装必要的包并设置环境。

In [ ]:
%pip install anthropic

In [ ]:
import anthropic
import os

# 将API密钥设置为环境变量或直接设置
# os.environ["ANTHROPIC_API_KEY"] = "your-api-key-here"

# 初始化客户端
client = anthropic.Anthropic()

# 辅助函数
def print_thinking_response(response):
    """美观地打印带有思考块的消息响应。"""
    print("\n==== 完整响应 ====")
    for block in response.content:
        if block.type == "thinking":
            print("\n🧠 思考块:")
            # 为可读性显示截断的思考内容
            print(block.thinking[:500] + "..." if len(block.thinking) > 500 else block.thinking)
            print(f"\n[签名可用: {bool(getattr(block, 'signature', None))}]")
            if hasattr(block, 'signature') and block.signature:
                print(f"[签名（前50个字符）: {block.signature[:50]}...]")
        elif block.type == "redacted_thinking":
            print("\n🔒 编辑过的思考块:")
            print(f"[数据长度: {len(block.data) if hasattr(block, 'data') else 'N/A'}]")
        elif block.type == "text":
            print("\n✓ 最终答案:")
            print(block.text)
    
    print("\n==== 响应结束 ====")

def count_tokens(messages):
    """计算给定消息列表的令牌数。"""
    result = client.messages.count_tokens(
        model="claude-sonnet-4-5",
        messages=messages
    )
    return result.input_tokens

## 基本示例

让我们从一个基本示例开始，展示扩展思考的作用：

In [ ]:
def basic_thinking_example():
    response = client.messages.create(
        model="claude-sonnet-4-5",
        max_tokens=4000,
        thinking= {
            "type": "enabled",
            "budget_tokens": 2000
        },
        messages=[{
            "role": "user",
            "content": "Solve this puzzle: Three people check into a hotel. They pay $30 to the manager. The manager finds out that the room only costs $25 so he gives $5 to the bellboy to return to the three people. The bellboy, however, decides to keep $2 and gives $1 back to each person. Now, each person paid $10 and got back $1, so they paid $9 each, totaling $27. The bellboy kept $2, which makes $29. Where is the missing $1?"
        }]
    )
    
    print_thinking_response(response)

basic_thinking_example()


==== FULL RESPONSE ====

🧠 THINKING BLOCK:
Let's work through this problem step by step:

Initial situation:
- Three people each pay $10, for a total of $30 given to the manager.
- The room actually costs $25.
- Manager gives $5 to the bellboy to return to the customers.
- Bellboy keeps $2 and gives $1 back to each person ($3 total).

After these transactions:
- Each person has effectively paid $9 (they paid $10 and got $1 back).
- So the three people together paid $27.
- The hotel kept $25 for the room.
- The bellboy kept $2.

So the mo...

[Signature available: True]
[Signature (first 50 chars): EuYBCkQYAiJAGF6X7aWRuRByTdymAUdNOMC++3ZqSJv7jcY5Ly...]

✓ FINAL ANSWER:
# Hotel Bill Puzzle Solution

This is a classic misdirection puzzle that confuses us by mixing up two different accounting approaches.

## The actual flow of money

1. Three people each pay $10, totaling $30
2. The hotel keeps $25 for the room
3. The bellboy keeps $2
4. The guests receive $3 back ($1 each)

## The accoun

## 使用扩展思考进行流式处理

此示例展示如何处理带有思考的流式传输：

In [ ]:
def streaming_with_thinking():
    with client.messages.stream(
        model="claude-sonnet-4-5",
        max_tokens=4000,
        thinking={
            "type": "enabled",
            "budget_tokens": 2000
        },
        messages=[{
            "role": "user",
            "content": "Solve this puzzle: Three people check into a hotel. They pay $30 to the manager. The manager finds out that the room only costs $25 so he gives $5 to the bellboy to return to the three people. The bellboy, however, decides to keep $2 and gives $1 back to each person. Now, each person paid $10 and got back $1, so they paid $9 each, totaling $27. The bellboy kept $2, which makes $29. Where is the missing $1?"
        }]
    ) as stream:
        # 跟踪我们当前正在构建的内容
        current_block_type = None
        current_content = ""
        
        for event in stream:
            if event.type == "content_block_start":
                current_block_type = event.content_block.type
                print(f"\n--- 开始 {current_block_type} 块 ---")
                current_content = ""
                
            elif event.type == "content_block_delta":
                if event.delta.type == "thinking_delta":
                    print(event.delta.thinking, end="", flush=True)  # 只是打印思考的点以避免混乱
                    current_content += event.delta.thinking
                elif event.delta.type == "text_delta":
                    print(event.delta.text, end="", flush=True)
                    current_content += event.delta.text
                    
            elif event.type == "content_block_stop":
                if current_block_type == "thinking":
                    # 只是显示思考的摘要
                    print(f"\n[完成思考块，{len(current_content)} 个字符]")
                elif current_block_type == "redacted_thinking":
                    print("\n[编辑过的思考块]")
                print(f"--- 完成 {current_block_type} 块 ---\n")
                current_block_type = None
                
            elif event.type == "message_stop":
                print("\n--- 消息完成 ---")

streaming_with_thinking()

## 令牌计数和上下文窗口管理

此示例演示如何使用扩展思考跟踪令牌使用情况：

In [ ]:
def token_counting_example():
    # 定义创建示例提示的函数
    def create_sample_messages():
        messages = [{
            "role": "user",
            "content": "Solve this puzzle: Three people check into a hotel. They pay $30 to the manager. The manager finds out that the room only costs $25 so he gives $5 to the bellboy to return to the three people. The bellboy, however, decides to keep $2 and gives $1 back to each person. Now, each person paid $10 and got back $1, so they paid $9 each, totaling $27. The bellboy kept $2, which makes $29. Where is the missing $1?"
        }]
        return messages
    
    # 计算不带思考的令牌数
    base_messages = create_sample_messages()
    base_token_count = count_tokens(base_messages)
    print(f"基础令牌计数（仅输入）: {base_token_count}")
    
    # 发送带有思考的请求并检查实际使用情况
    response = client.messages.create(
        model="claude-sonnet-4-5",
        max_tokens=8000,
        thinking = {
            "type": "enabled",
            "budget_tokens": 2000
        },
        messages=base_messages
    )
    
    # 计算并打印令牌使用统计
    thinking_tokens = sum(
        len(block.thinking.split()) * 1.3  # 粗略估计
        for block in response.content 
        if block.type == "thinking"
    )
    
    final_answer_tokens = sum(
        len(block.text.split()) * 1.3  # 粗略估计
        for block in response.content 
        if block.type == "text"
    )
    
    print(f"\n估计使用的思考令牌: ~{int(thinking_tokens)}")
    print(f"估计最终答案令牌: ~{int(final_answer_tokens)}")
    print(f"估计总输出令牌: ~{int(thinking_tokens + final_answer_tokens)}")
    print(f"输入令牌 + max_tokens = {base_token_count + 8000}")
    print(f"思考后可用于最终答案的: ~{8000 - int(thinking_tokens)}")
    
    # 演示递增思考预算
    thinking_budgets = [1024, 2000, 4000, 8000, 16000, 32000]
    context_window = 200000
    for budget in thinking_budgets:
        print(f"\n思考预算为 {budget} 令牌时:")
        print(f"输入令牌: {base_token_count}")
        print(f"所需最大令牌: {base_token_count + budget + 1000}")  # 为最终答案添加1000
        print(f"剩余上下文窗口: {context_window - (base_token_count + budget + 1000)}")
        
        if base_token_count + budget + 1000 > context_window:
            print("警告：这将超过200k令牌的上下文窗口！")

# 取消注释以运行示例
token_counting_example()

## 理解编辑过的思考块

偶尔，Claude的内部推理会被安全系统标记。当发生这种情况时，我们会加密部分或全部的 `thinking` 块，并将其作为 `redacted_thinking` 块返回给您。这些编辑过的思考块在传递回API时会解密，允许Claude在不丢失上下文的情况下继续其响应。

此示例演示使用触发编辑思考的特殊测试字符串来处理编辑过的思考块：

In [ ]:
def redacted_thinking_example():
    # 使用触发编辑思考的特殊测试字符串
    response = client.messages.create(
        model="claude-sonnet-4-5",
        max_tokens=4000,
        thinking={
            "type": "enabled",
            "budget_tokens": 2000
        },
        messages=[{
            "role": "user",
            "content": "ANTHROPIC_MAGIC_STRING_TRIGGER_REDACTED_THINKING_46C9A13E193C177646C7398A98432ECCCE4C1253D5E2D82641AC0E52CC2876CB"
        }]
    )
    
    # 识别编辑过的思考块
    redacted_blocks = [block for block in response.content if block.type == "redacted_thinking"]
    thinking_blocks = [block for block in response.content if block.type == "thinking"]
    text_blocks = [block for block in response.content if block.type == "text"]
    print(response.content)
    print(f"响应包含 {len(response.content)} 个总块:")
    print(f"- {len(redacted_blocks)} 个编辑过的思考块")
    print(f"- {len(thinking_blocks)} 个常规思考块")
    print(f"- {len(text_blocks)} 个文本块")
    
    # 显示编辑块的数据属性
    if redacted_blocks:
        print(f"\n编辑过的思考块包含加密数据:")
        for i, block in enumerate(redacted_blocks[:3]):  # 最多显示前3个
            print(f"块 {i+1} 数据预览: {block.data[:50]}...")
    
    # 打印最终文本输出
    if text_blocks:
        print(f"\n最终文本响应:")
        print(text_blocks[0].text)

# 取消注释以运行示例
redacted_thinking_example()

## 处理错误情况

使用扩展思考时，请记住：

1. **最小预算**：最小思考预算是1,024个令牌。我们建议从最小值开始，然后递增增加以找到最佳范围。

2. **不兼容的功能**：思考与温度、top_p或top_k修改不兼容，您也无法预填响应。

3. **定价**：扩展思考令牌计入上下文窗口并按输出令牌计费。它们也计入您的速率限制。

有关使用工具的扩展思考的更多详细信息，请参阅"使用工具的扩展思考"笔记本。

In [ ]:
def demonstrate_common_errors():
    # 1. 思考预算设置过小导致的错误
    try:
        response = client.messages.create(
            model="claude-sonnet-4-5",
            max_tokens=4000,
            thinking={
                "type": "enabled",
                "budget_tokens": 500  # 太小，最小值是1024
            },
            messages=[{
                "role": "user",
                "content": "Explain quantum computing."
            }]
        )
    except Exception as e:
        print(f"\n思考预算过小的错误: {e}")
    
    # 2. 将温度与思考一起使用导致的错误
    try:
        response = client.messages.create(
            model="claude-sonnet-4-5",
            max_tokens=4000,
            temperature=0.7,  # 与思考不兼容
            thinking={
                "type": "enabled",
                "budget_tokens": 2000
            },
            messages=[{
                "role": "user",
                "content": "Write a creative story."
            }]
        )
    except Exception as e:
        print(f"\n温度和思考的错误: {e}")
    
    # 3. 超出上下文窗口导致的错误
    try:
        # 创建一个非常大的提示
        long_content = "Please analyze this text. " + "This is sample text. " * 150000
        
        response = client.messages.create(
            model="claude-sonnet-4-5",
            max_tokens=20000,  # 这加上长提示将超出上下文窗口
            thinking={
                "type": "enabled",
                "budget_tokens": 10000
            },
            messages=[{
                "role": "user",
                "content": long_content
            }]
        )
    except Exception as e:
        print(f"\n超出上下文窗口的错误: {e}")

# 运行常见错误示例
demonstrate_common_errors()